In [ ]:
# Intall the library to access arkindex exports files https://doc.teklia.com/arkindex_export/
!pip install arkindex-export


  Using cached peewee-3.17.0.tar.gz (2.9 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for peewee: filename=peewee-3.17.0-py3-none-any.whl size=135807 sha256=484594447fbd6b711edd4fb9dfcfe2181408c9c88b38649abab38799b71bdf09
  Stored in directory: /Users/ckermorvant/Library/Caches/pip/wheels/2e/e5/bf/da60fe7ec992ef3654dd8d4341b126e808e0e7d14c54253180
Successfully built peewee
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [arkindex-export]


In [3]:
from arkindex_export import open_database, Element, Metadata
from arkindex_export.queries import list_children
from pathlib import Path

# load the  export
open_database(Path("cke-atelier-pictoria-18092025-20251127-090658.sqlite"))


In [5]:
# compute some statistics
print("Number of folders", Element.select().where(Element.type == 'folder').count())
print("Number of pages:", Element.select().where(Element.type == 'page').count())
print("Number of photographs:", Element.select().where(Element.type == 'photograph').count())
print("Number of persons:", Element.select().where(Element.type == 'person').count())
print("Number of metadata:", Metadata.select().count())
print("Number of metadata Title:", Metadata.select().where(Metadata.name == 'Title').count())
print("Number of metadata Creator:", Metadata.select().where(Metadata.name == 'Creator').count())


Number of folders 2
Number of pages: 191
Number of photographs: 84
Number of persons: 0
Number of metadata: 225
Number of metadata Title: 2
Number of metadata Creator: 5


In [9]:
# count the number of pages in each folder
for folder in Element.select().where(Element.type == 'folder'):
    page_number = list_children(folder.id).where(Element.type == 'page').count()
    print(f"Folder '{folder.name}' has {page_number} pages.")

Folder 'BnF, département Société de Géographie, SG WD-346' has 100 pages.
Folder 'BnF, département Société de Géographie, SG WD-232 (RES)' has 91 pages.


In [ ]:
#!pip install shapely
from shapely import Polygon
import ast
# compute the max, min and average size of photographs
# loop on all pages and get the size of each photograph

# for the documentation about element objects, see the code :
#  https://gitlab.teklia.com/arkindex/export/-/blob/master/arkindex_export/models.py?ref_type=heads#L86

sizes = []
for page in Element.select().where(Element.type == 'page'):
    photographs = list_children(page.id).where(Element.type == 'photograph')
    for photo in photographs:
        # convert photo.polygon to shapely polygon
        polygon = Polygon(ast.literal_eval(photo.polygon))
        minx, miny, maxx, maxy = polygon.bounds
        width = maxx - minx
        height = maxy - miny
        sizes.append((width, height))
# compute max, min and average size
widths, heights = zip(*sizes)
print("Photograph width - min:", min(widths), "max:", max(widths), "avg:", sum(widths)/len(widths))
print("Photograph height - min:", min(heights), "max:", max(heights), "avg:", sum(heights)/len(heights))    


Photograph width - min: 2915.0 max: 5761.0 avg: 3361.4285714285716
Photograph height - min: 2367.0 max: 6133.0 avg: 3570.1666666666665
